In [ ]:
# set up a connection to SQL Server and pull some data 
# first, import the required package and define parameters 

import pyodbc  # this is the built-in package for making ODBC connections

# driver
drv = 'ODBC Driver 17 for SQL Server'
# server
svr = 'vhacdwrb02.vha.med.va.gov'
# database
db = 'ORD_Davies_202110040D'
# schema
sch = 'Dflt'
# table 
tbl = 'rad_rows_kept'

incols = '''
patientICN, examDateTime, radsrc, RadNucMedReportSID, lungdxdt_minus_repdt, age, sentID, item, dim1mm, dim2mm, dim3mm, sent
'''

In [ ]:
# read SQL table created by notebook read_rad_notes into a Pandas dataframe called 'indata'
import pandas as pd
import numpy as np

conn = pyodbc.connect('Driver={%s};Server=%s;Database=%s;Trusted_Connection=yes;' % (drv, svr, db))
cursor = conn.cursor()

qry = 'select %s from [%s].[%s].[%s]' % (incols, db, sch, tbl)

indata = pd.read_sql(qry, conn)

conn.close()

# calculate max nodule size for each record, sort by patient / exam date, and take first record for each pt
indata['repdate'] = pd.to_datetime(indata['examDateTime']).dt.date

# for now, reset nodule dimensions > 75 mm to missing - this might be too restrictive
vars = ['dim1mm', 'dim2mm', 'dim3mm']
for v in vars:
    indata.loc[indata[v] > 75, v] = np.NaN


indata['max_mm'] = indata[['dim1mm', 'dim2mm', 'dim3mm']].max(axis = 1)

print(indata.head())

In [ ]:
import datetime

preonly = indata.loc[(indata['lungdxdt_minus_repdt']>=0) & (indata['lungdxdt_minus_repdt']<90)]
# & (indata['repdate']>=datetime.date(2018, 4, 1))]

preonly.sort_values(by = ['patientICN', 'examDateTime'], ascending = [True, False], inplace = True)

#indata.sort_values(by = ['patientICN', 'examDateTime'], ascending = [True, True], inplace = True)

#firstrep = indata.groupby('patientICN').nth(0)
firstrep = preonly.groupby('patientICN').nth(0)

print(len(firstrep.index))

In [ ]:
# keep just columns of interest and do some re-naming
forplot = firstrep[['examDateTime', 'max_mm', 'lungdxdt_minus_repdt', 'age']].rename(columns = {'lungdxdt_minus_repdt':'lead_time'})

#forplot.set_index('examDateTime')
#forplot.drop(columns = ['patientICN'])
#forplot.rename(columns={'lungdxdt_minus_repdt':'lead_time'}, inplace = True)
print(forplot.head())

In [ ]:
formodel = forplot.copy()

formodel['isPan'] = 0
formodel.loc[formodel['examDateTime'] >= datetime.datetime(2020, 3, 1), 'isPan'] = 1

def cw(row):
    if \
       (
        row['examDateTime'] >= datetime.datetime(2019, 3, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2019, 6, 1) \
        ) \
    or \
       (
        row['examDateTime'] >= datetime.datetime(2020, 3, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2020, 6, 1) \
        ):
        val = 1
        
    elif \
       (
        row['examDateTime'] >= datetime.datetime(2019, 6, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2019, 9, 1) \
        ) \
    or \
       (
        row['examDateTime'] >= datetime.datetime(2020, 6, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2020, 9, 1) \
        ):
        val = 2
        
    elif \
       (
        row['examDateTime'] >= datetime.datetime(2019, 9, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2019, 12, 1) \
        ) \
    or \
       (
        row['examDateTime'] >= datetime.datetime(2020, 9, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2020, 12, 1) \
        ):
        val = 3
        
    elif \
       (
        row['examDateTime'] >= datetime.datetime(2019, 12, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2020, 3, 1) \
        ) \
    or \
       (
        row['examDateTime'] >= datetime.datetime(2020, 12, 1) \
        and \
        row['examDateTime'] < datetime.datetime(2021, 3, 1) \
        ):
        val = 4
    else:
        val = np.NaN
  
    return val

formodel['compwin'] = formodel.apply(cw, axis = 1)    

print (formodel.head())



In [ ]:
import scipy
#dir(scipy.stats)
import statsmodels.api as sm
import statsmodels.formula.api as smf 
from matplotlib import pyplot as plt

print (dir(sm.families))

In [ ]:
allmm = formodel[['max_mm']].to_numpy()

hist, bins = np.histogram(allmm, range = (1, 75))
#print (hist)

plt.hist(allmm, bins = bins)

In [ ]:
help(smf.glm)

In [ ]:
model = 'max_mm ~ isPan+age+lead_time'

wind = {
    1: 'Mar-May',
    2: 'Jun-Aug',
    3: 'Sep-Nov',
    4: 'Dec-Feb'
}

def winmodel(win):
    windf = formodel.loc[formodel['compwin'] == win]
    #print (windf.head())
    mod = smf.glm(formula = model, data = windf, family = sm.families.Gamma())
    result = mod.fit()
    print ('\n\n********** RESULTS FOR WINDOW: %s ***********\n' % wind[win])
    print ('\n...summary:\n')
    print (result.summary())
    print ('\n...coefficients:\n')
    print (result.params)
    print ('\n...p-values:\n')
    print (result.pvalues)

In [ ]:
winmodel(1)
winmodel(2)
winmodel(3)
winmodel(4)

In [ ]:
# calculate weekly means and counts and re-name the index to 'weekStarting'
byweek = forplot.resample('1W', on = 'examDateTime').mean()
byweek_count = forplot.resample('1W', on = 'examDateTime').agg({'max_mm':'count'}).rename(columns = {'max_mm':'nppl'})

print (byweek.head())
print (byweek_count.head())

byweek = pd.merge(
    byweek,
    byweek_count,
    left_on = ['examDateTime'],
    right_on = ['examDateTime'],
    how = 'inner'
)

byweek.index.names = ['weekStarting']

del byweek_count
print (byweek.head())

In [ ]:
# calculate 12-week weighted moving averages 
byweek['sp_mm'] = byweek['max_mm'] * byweek['nppl']
byweek['sp_lt'] = byweek['lead_time'] * byweek['nppl']
byweek['sp_ag'] = byweek['age'] * byweek['nppl']

rollingavg = byweek[['sp_mm', 'sp_lt', 'sp_ag', 'nppl']].rolling(12).sum()
rollingavg['sp_mm'] = rollingavg['sp_mm'] / rollingavg['nppl']
rollingavg['sp_lt'] = rollingavg['sp_lt'] / rollingavg['nppl']
rollingavg['sp_ag'] = rollingavg['sp_ag'] / rollingavg['nppl']
rollingavg['sp_nppl'] = rollingavg['nppl'] / 12
rollingavg['sp_nppl_qc'] = byweek[['nppl']].rolling(12).mean()

byweek = pd.merge(
    byweek[['max_mm', 'lead_time', 'age', 'nppl']],
    rollingavg[['sp_mm', 'sp_lt', 'sp_ag', 'sp_nppl', 'sp_nppl_qc']],
    left_on = ['weekStarting'],
    right_on = ['weekStarting'],
    how = 'left'
)

print (byweek.head(30))

In [ ]:
# make a plot...
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
byweek2 = byweek.loc[datetime.date(2018,4,1):]
byweek2.reset_index(inplace = True)

#byweek2 = byweek.reset_index(inplace = False)
print(byweek2.head())

ws = byweek2[['weekStarting']].to_numpy()
mm = byweek2[['max_mm']].to_numpy()
lt = byweek2[['lead_time']].to_numpy()
ag = byweek2[['age']].to_numpy()
n = byweek2[['nppl']].to_numpy()
mm12 = byweek2[['sp_mm']].to_numpy()
lt12 = byweek2[['sp_lt']].to_numpy()
ag12 = byweek2[['sp_ag']].to_numpy()
n12 = byweek2[['sp_nppl']].to_numpy()



In [ ]:
#ws[17]
panstart = np.array(['2020-03-01T00:00:00.000000000'], dtype='datetime64[ns]')

print (min(mm))
np.where(np.in1d(ws, panstart))[0][0]

#np.where(ws == ['2020-03-01T00:00:00.000000000'])[0]


In [ ]:
dtlabs = pd.date_range(start = '20180301', end = '20220228', freq = 'W').strftime('%Y%m%d')
#print(dtlabs[:6])
dts = [d for i, d in enumerate(dtlabs) if i % 4 == 0]
#print (dts[:6])

In [ ]:
fig, ax1 = plt.subplots(figsize = (16, 6))

ax2 = ax1.twinx()
#print (type(fig), type(ax1), type(ax2))
ax1.plot(ws, mm, color = 'red', linestyle = 'dotted', alpha = 0.2)
ax2.plot(ws, n, color = 'black', linestyle = 'dotted', alpha = 0.2)

ax1.plot(ws, mm12, color = 'red', linestyle = 'solid')
ax2.plot(ws, n12, color = 'black', linestyle = 'solid')

ax1.vlines(x = panstart, ymin = min(mm), ymax = max(mm), colors = 'black', 
           label = 'pandemic start', linestyles = 'dashed')
ax1.set_xlabel('week')
ax1.set_ylabel('mean nodule size (mm)')
ax1.yaxis.label.set_color('red')
ax2.set_ylabel('# of incident lung ca cases')
ax2.yaxis.label.set_color('black')

ax1.set_xticks(dts)
ax1.set_xticklabels(dts, rotation = 90, ha = 'center')

fig.suptitle('mean lung nodule size on incident scan, by week, Mar 2018-Feb 2022')
fig.autofmt_xdate(rotation = 90)

In [ ]:
figb, ax1b = plt.subplots(figsize = (16, 6))

ax2b = ax1b.twinx()
#print (type(fig), type(ax1), type(ax2))
ax1b.plot(ws, mm, color = 'red', linestyle = 'dotted', alpha = 0.2)
ax2b.plot(ws, ag, color = 'black', linestyle = 'dotted', alpha = 0.2)

ax1b.plot(ws, mm12, color = 'red', linestyle = 'solid')
ax2b.plot(ws, ag12, color = 'black', linestyle = 'solid')

ax1b.vlines(x = panstart, ymin = min(mm), ymax = max(mm), colors = 'black', 
           label = 'pandemic start', linestyles = 'dashed')
ax1b.set_xlabel('week')
ax1b.set_ylabel('mean nodule size (mm)')
ax1b.yaxis.label.set_color('red')
ax2b.set_ylabel('mean age at scan')
ax2b.yaxis.label.set_color('black')

ax1b.set_xticks(dts)
ax1b.set_xticklabels(dts, rotation = 90, ha = 'center')

figb.suptitle('mean lung nodule size on incident scan, by week, Mar 2018-Feb 2022')
figb.autofmt_xdate(rotation = 90)

In [ ]:
# 'rb': right bound of 12-week date range
# select 4 pre-pandemic windows and 4 pandemic windows with the same seasonality

#pre1_rb = np.array(['2019-03-03T00:00:00.000000000'], dtype='datetime64[ns]')
pre2_rb = np.array(['2019-05-26T00:00:00.000000000'], dtype='datetime64[ns]')
pre3_rb = np.array(['2019-08-18T00:00:00.000000000'], dtype='datetime64[ns]')
pre4_rb = np.array(['2019-11-10T00:00:00.000000000'], dtype='datetime64[ns]')
pre5_rb = np.array(['2020-02-02T00:00:00.000000000'], dtype='datetime64[ns]')

#pst1_rb = np.array(['2019-03-03T00:00:00.000000000'], dtype='datetime64[ns]')
pst2_rb = np.array(['2020-05-24T00:00:00.000000000'], dtype='datetime64[ns]')
pst3_rb = np.array(['2020-08-16T00:00:00.000000000'], dtype='datetime64[ns]')
pst4_rb = np.array(['2020-11-08T00:00:00.000000000'], dtype='datetime64[ns]')
pst5_rb = np.array(['2021-01-31T00:00:00.000000000'], dtype='datetime64[ns]')

pre = [pre2_rb, pre3_rb, pre4_rb, pre5_rb]
pst = [pst2_rb, pst3_rb, pst4_rb, pst5_rb]

pre_x = []
pst_x = []

for w in pre:
    pre_x.append(np.where(np.in1d(ws, w))[0][0])

for w in pst:
    pst_x.append(np.where(np.in1d(ws, w))[0][0])

print(pre_x)
print(pst_x)

#ws[17]
#panstart = np.array(['2020-03-01T00:00:00.000000000'], dtype='datetime64[ns]')

#print (min(mm))
#np.where(np.in1d(ws, panstart))[0][0]

In [ ]:
import scipy
#dir(scipy.stats)
import statsmodels.api as sm
import statsmodels.formula.api as smf 

In [ ]:
print(byweek.head())
plt.plot(byweek.sp_mm)

In [ ]:
fig, ax1 = plt.subplots(figsize = (12, 6))
ax2 = ax1.twinx()

print (byweek.columns)

In [ ]:
#wkticks = [pd.datetime(x[0]).dt.date for i, x in enumerate(list(ws)) if i % 2 == 0]
#print(wkticks)
#pd.datetime(ws[0][0]).dt.date

In [ ]:
ax1 = byweek.plot(kind = 'line', x = 'weekStarting', y = 'sp_mm', color = 'Red')
ax2 = byweek.plot(kind = 'line', x = 'weekStarting', y = 'sp_lt', color = 'Black',
                 secondary_y = True, ax = ax1)

fig = plt.figure()

fig.title('mean lung nodule size on incident scan, by week, Mar 2018-Feb 2022')

ax1.set_xlabel('week')
ax1.set_ylabel('mean nodule size (mm)')
ax2.set_ylabel('mean lead time (days)')

fig.tight_layout()

fig.show()
